In [4]:
import numpy as np
import pandas as pd 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [6]:
df = pd.read_csv('/kaggle/input/c/titanic/train.csv')

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<h1>Using pipelines</h1>

In [8]:
#removing un useful columns
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True) #inplace is directly making changes in df

In [9]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [10]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [11]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [12]:
#imputation transformer:
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [13]:
#one hot encoding:
trf2 = ColumnTransformer(
    transformers=[
        ("ohe_sex_embarked", OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
    ],
    remainder='passthrough'
)

In [14]:
#scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)), #slice func do the transformation on all columns given as the parameter
])

In [15]:
#feature selection:
trf4 = SelectKBest(score_func=chi2,k=8) #using the top 5 most important columns

In [16]:
#Train the model
trf5 = DecisionTreeClassifier()

<h1>Create Pipeline</h1>

In [17]:
pipe = Pipeline([
    ('trf1', trf1),  # imputation
    ('trf2', trf2),  # one hot encoding
    ('trf3', trf3),  # scaling
    ('trf4', trf4),  # feature selection
    ('trf5', trf5)   # model
])

<h1>pipeline vs make_pipeline</h1>
<p1>which one to use??</p1><br>
<p1>we should use the make pipeline instead of directly calling the Pipeline class bcs the syntax is much more easier that way</p1><br><p1>Same as columtransformer vs make_columntransformer</p1>

In [18]:
#alternate code

#pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [19]:
#Training:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7c6568816160>)),
                ('trf5', DecisionTreeClassifier())])

<h1>Exploring the pipeline</h1>

In [20]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7c6568816160>),
 'trf5': DecisionTreeClassifier()}

In [21]:
#display 
from sklearn import set_config
set_config(display='diagram')

In [22]:
#predict
y_pred = pipe.predict(x_test)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

<h1>Cross validation using pipeline</h1>

In [26]:
# cross validation using cross_val score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean()

0.6391214419383433

<h1>Gridsearch using pipelines</h1>

In [31]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None] #hyper parameter tuning 
}

In [32]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7c6568816160>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [33]:
grid.best_score_

0.6391214419383433

In [34]:
grid.best_params_

{'trf5__max_depth': 2}

<h1>Exporting the pipeline</h1>

In [35]:
import pickle
import os 
os.makedirs('models', exist_ok=True) #check if directory exists pickle does not create automatically
pickle.dump(pipe,open('pipe.pkl','wb'))